# Benchmarking Pyxis

The following notebook contains tests for benchmarking `pyxis` functions/commands for runtime and memory usage, both against example test files (several provided in the `example-files` directory) and also against a real [ChIP peaks ISL1 transcription factor dataset](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5838045) (provided in the `analysis` directory). 

There are also comparisons drawn to [HOMER](http://homer.ucsd.edu/homer/ngs/peakMotifs.html)'s analogous command, `findMotifsGenome.pl`.

In [25]:
# should cd into outer pyxis directory
%cd ..

/home/esquan/final_project


In [27]:
import numpy as np
import os
import pandas as pd
import pyfaidx
import pyxis.myutils
import random
import scipy.stats
import sys

In [28]:
!python setup.py install --user > path.txt 2> /dev/null
# determining where pyxis script installed to, and saving that path to export later
with open('path.txt', 'r') as file:
    for line in file:
        if 'Installing pyxis script to' in line:
            path = line.split()[-1]
            f = open("scriptpath.txt", "w")
            f.write(path)
            f.close()
            break

## I. Runtime

### Loading in the Example Data

In [29]:
# example-files directory test files
cwd = os.getcwd()
f_peaks_dir = cwd + '/example-files/peaks.bed'      # foreground peaks file
b_peaks_dir = cwd + '/example-files/background.bed' # background peaks file
ref_dir = cwd + '/example-files/ref.fa'             # reference genome file
pwms_dir = cwd + '/example-files/test.pwms'         # file of known PWMs

# index reference genome file
test_ref = pyfaidx.Fasta(ref_dir)

### Testing Runtime of Various myutil.py Functions

In [5]:
print("Runtime for ReadBED:")
%timeit readbed = pyxis.myutils.ReadBED(f_peaks_dir, test_ref)
print("\nRuntime for ReadPWMS:")
%timeit readpwms = pyxis.myutils.ReadPWMS(pwms_dir)
print("\nRuntime for WriteFastaSeq:")
%timeit pyxis.myutils.WriteFastaSeq(test_ref, 'chr6', 1, 100)
print("\nRuntime for ComputeNucFreqs:")
f_seqs = ['ACTAGCTACG', 'TAGCATGCTAGAC', 'TACTATCATGGGT', 'GTAGCTAGTACAGTACAGTAA', 'ATCAGTAC', 'TAGCTAGCTA', 'GTCAGTACGATAC']
%timeit pyxis.myutils.ComputeNucFreqs(f_seqs)

Runtime for ReadBED:
183 µs ± 9.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Runtime for ReadPWMS:
247 µs ± 6.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Runtime for WriteFastaSeq:
8.87 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Runtime for ComputeNucFreqs:
23.9 µs ± 709 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
print("\nRuntime for RandomBkSequence:")
seq_lens = [10, 13, 13, 21, 8, 10, 13]
%timeit bk_seqs = pyxis.myutils.RandomBkSequence(seq_lens, 7, test_ref)
print("\nRuntime for ScoreSeq:")
bach_pwm = np.array([
    [-0.705,-0.938,-1.27,1.041],
    [-2.694,-1.138,1.265,-2.694],
    [-1.681,1.298,-2.819,-2.394],
    [-1.138,-1.818,-3.126,1.245],
    [-1.421,-1.389,1.147,-1.022],
    [1.101,-1.818,-1.454,-0.516],
    [-1.681,-0.108,0.978,-1.358],
    [-2.961,-2.584,-3.126,1.343],
    [-1.389,1.294,-2.584,-3.573],
    [1.34,-2.819,-3.126,-2.584],
    [-1.454,0.244,-0.163,0.495]
])
%timeit score = pyxis.myutils.ScoreSeq(bach_pwm, f_seqs)
print("\nRuntime for ReverseComplement:")
test_seq = 'GTAGCTAGCATTACATTACTGTACGTAC'
%timeit pyxis.myutils.ReverseComplement(test_seq)
print("\nRuntime for FindMaxScore:")
%timeit pyxis.myutils.FindMaxScore(bach_pwm, test_seq)


Runtime for RandomBkSequence:
88.2 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Runtime for ScoreSeq:
1.48 µs ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Runtime for ReverseComplement:
6.29 µs ± 46.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Runtime for FindMaxScore:
141 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
PWMList = [np.array([[-1.83361299, -2.15554108, -2.31903981,  0.81189105, -6.96289593,
         -4.15554107, -1.67749378, -1.26245629, -0.14271704,  1.44649493,
         -1.792971  , -2.71496849, -2.3779335 ,  1.16122531,  1.15084616,
          1.62956103],
        [-0.43933405,  1.17153032,  1.12456684, -4.37793349, -3.50346438,
         -6.96289593, -0.71496849, -6.96289593, -0.00869969, -1.67749378,
         -4.6409679 , -2.10491501,  1.34544303, -1.29047066, -1.40830715,
         -2.71496849],
        [ 0.22692855, -0.81314888, -3.962896  , -4.96289599, -4.6409679 ,
         -5.96289597,  1.57626281, -3.3779335 , -0.67749379, -1.91850188,
         -2.71496849,  1.58785078, -1.08025295, -2.87543316, -0.792971  ,
         -1.71496849],
        [ 0.85728296, -0.06807824,  0.63701684,  1.11391959,  1.94999333,
          1.97079465, -3.3779335 ,  1.79865523,  0.56066595, -0.51995251,
          1.81518112, -0.71496849, -0.33353938,  0.28503151, -0.27639548,
         -1.15554108]]),
np.array([[  0.55777767,  -0.41119543,   1.40925515,  -5.96578425,
           1.97085365,  -4.96578427,   1.9441088 ,   1.87656606,
          -1.41119543,   1.18903382],
        [ -0.96578428,   0.78240857,  -4.96578427, -31.21928095,
          -5.96578425,  -3.50635266,  -5.96578425,  -6.96578421,
          -0.45798964,  -0.81603716],
        [  0.31961793,   0.18396284,  -3.96578428,   1.99421765,
          -4.96578427,  -2.64385619,  -5.96578425,  -1.83650127,
           1.48129894,   0.03421572],
        [ -0.38082178,  -1.35107444,   0.31961793, -31.21928095,
          -4.96578427,   1.89530262,  -3.05889368,  -4.64385618,
          -3.26534456,  -2.96578428]]),
 np.array([[ 1.73767068e-01,  8.67105730e-01, -1.79585928e+00,
         -4.57989644e-01, -7.36965593e-01, -4.96578427e+00,
          1.92599942e+00,  1.75872957e+00,  1.88283866e+00,
         -4.96578427e+00, -3.96578428e+00, -2.57346686e+00,
         -3.79585928e+00,  1.48954294e+00],
        [ 2.29004027e-02, -2.96578428e+00, -3.79585928e+00,
         -1.83650127e+00,  3.01002257e-01,  1.88908410e+00,
         -3.50635266e+00, -4.96578427e+00, -3.12192809e+01,
         -5.38082176e+00, -4.96578427e+00,  9.88412026e-01,
          1.74157485e+00, -1.96578428e+00],
        [ 5.01821266e-01,  6.70840336e-01,  1.76213617e+00,
          3.82943870e-01,  1.53156789e-01, -4.64385618e+00,
         -3.96578428e+00, -7.95859282e-01, -1.87832144e+00,
          1.95307895e+00,  6.11644544e-01, -9.65784284e-01,
         -3.96578428e+00, -9.43416471e-01],
        [-1.18442457e+00, -1.13289427e+00, -2.01158797e+00,
          7.55314904e-01,  7.86098352e-02, -2.15842936e+00,
         -4.38082177e+00, -6.96578421e+00, -4.64385618e+00,
         -3.79585928e+00,  1.24853484e+00,  4.17920008e-01,
         -9.43416471e-01, -1.26534457e+00]])]

In [9]:
print("\nRuntime for RandomSequence:")
test_freqs = [0.15, 0.34, 0.26, 0.25]
%timeit pyxis.myutils.RandomSequence(50, test_freqs)
print("\nGetThreshold:")
numsim = 10000
pval = 0.01
for i in range(3):
    null_scores = [pyxis.myutils.ScoreSeq(PWMList[i], pyxis.myutils.RandomSequence(PWMList[i].shape[1], test_freqs)) for j in range(numsim)]
%timeit pyxis.myutils.GetThreshold(null_scores, pval)
print("\nRuntime for ComputeEnrichment:")
%timeit pyxis.myutils.ComputeEnrichment(50, 20, 50, 25)
print("\nRuntime for pwm_to_ppm:")
%timeit pyxis.myutils.pwm_to_ppm(bach_pwm, test_freqs, 1e-10)


Runtime for RandomSequence:
23.1 µs ± 2.67 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

GetThreshold:
637 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Runtime for ComputeEnrichment:
2.15 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Runtime for pwm_to_ppm:
29.5 µs ± 496 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Summary:
Using small toy data and example test files provided in `example-files` directory:

| Function | Mean Runtime Per Loop ± Std. Dev. of 7 Runs (10000 loops each) |
| :------- | :-----------------------------------------------------: |
| ReadBED | 183 µs ± 9.18 µs |
| ReadPWMS | 247 µs ± 6.29 µs |
| WriteFastaSeq | 8.87 µs ± 139 ns |
| ComputeNucFreqs | 23.9 µs ± 709 ns |
| RandomBkSequence | 88.2 µs ± 1.15 µs |
| ScoreSeq | 1.48 µs ± 14.5 ns |
| ReverseComplement | 6.29 µs ± 46.8 ns |
| FindMaxScore | 141 µs ± 2 µs |
| RandomSequence | 23.1 µs ± 2.67 µs |
| GetThreshold | 637 µs ± 3.14 µs |
| ComputeEnrichment | 2.15 ms ± 280 µs |
| pwm_to_ppm | 29.5 µs ± 496 ns |

### Testing Pyxis Commands with Example Data

In [23]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

printf 'Basic Usage Command:\n'
python -m timeit -n 10 -s 'import os' 'os.system("pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms > /dev/null")'
printf '\nAdding a Background:\n'
python -m timeit -n 10 -s 'import os' 'os.system("pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms -b example-files/background.bed > /dev/null")'
printf '\nIncluding a specified p-value:\n'
python -m timeit -n 10 -s 'import os' 'os.system("pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms -p 0.005 > /dev/null")'

Basic Usage Command:
10 loops, best of 5: 1.97 sec per loop

Adding a Background:
10 loops, best of 5: 2.02 sec per loop

Including a specified p-value:
10 loops, best of 5: 2.03 sec per loop


In [33]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

printf 'Including generating motif sequence logos:\n'
python -m timeit -n 10 -s 'import os' 'os.system("pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms -s > /dev/null")'
printf '\nIncluding pseudovalue for generating sequence logos:\n'
python -m timeit -n 10 -s 'import os' 'os.system("pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms -s --pseudo 1e-6 > /dev/null")'

Generating motif sequence logos:
10 loops, best of 5: 2.82 sec per loop

Including pseudovalue for generating sequence logos:
10 loops, best of 5: 2.78 sec per loop


#### Summary:

| Command | Mean Runtime Per Loop (10 loops, best of 5) |
| :------- | :-----------------------------------------------------: |
| Basic Usage | 1.97 sec |
| Basic Usage + Background Provided | 2.02 sec |
| Basic Usage + P-value Provided | 2.03 sec |
| Basic Usage + Seqlogo | 2.82 sec |
| Basic Usage + Seqlogo + Pseudovalue | 2.78 sec |

### Testing Pyxis Commands with a Larger Dataset

This larger, real-life dataset we will test on `pyxis` is provided at GEO accession [GSM5838045](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5838045) (ISL1_WT_d6CP_Rep1) contains **peaks for the ISL1 transcription factor**. The ISL1 TFs were pulled down during ChIP through an anti-human-ISLET-1 antibody. This dataset is part of the [GSE195476 series](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE195476).

The genome build used for this dataset was **hg19**. Due to its high memory usage, the build itself will not be included in the `pyxis` repo. If you are interested in running the below commands with the ISL1 dataset (as well as the commands in `PyxisAnalysis.ipynb`), please install `hg19.fa.gz` [here](https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/) and move it into the `benchmark` directory of `pyxis`. Be sure to also decompress the file before running `pyxis` on it– you can do this in Linux with `gzip -d`!

The PWMs file serving as our motif database for this demonstration has position weight matrices selected based off Figure 3D from the source paper: [The multi-lineage transcription factor ISL1 controls cardiomyocyte cell fate through interaction with NKX2.5](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10679653/). Several other motifs picked at random from [HOCOMOCO](https://hocomoco11.autosome.org/) are also included for testing purposes.

In [19]:
pwd # Check that you are still in the outer pyxis directory

'/home/esquan/final_project/pyxis'

In [34]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

# Checking results from pyxis for ISL1 peaks dataset
pyxis analysis/GSM5838045_S1_ISL1_WT_d6CP_Rep1.bed benchmark/hg19.fa benchmark/isl1.pwms > /dev/null
cat pyxis_enrichments.tsv

motif_name	pval	log_pval	num_peak_motif	pct_peak_motif	num_bg_motif	pct_bg_motif	enriched_status
ISL1_MOUSE.H11MO.0.A	3.8681244406275597e-295	-294.4125	4835	0.781856403622251	2879	0.4655562742561449	Sig
NKX25_MOUSE.H11MO.0.A	6.859704709383046e-281	-280.16369	4801	0.7763583441138422	2890	0.46733505821474774	Sig
LHX3_MOUSE.H11MO.0.C	1.0008831183450168e-267	-266.99962	4212	0.6811125485122898	2286	0.36966364812419145	Sig
NKX22_MOUSE.H11MO.0.A	4.799427573329636e-218	-217.31881	5292	0.8557567917205692	3761	0.6081824062095731	Sig
BACH2_HUMAN.H11MO.0.A	4.856501062484705e-217	-216.31368	4781	0.773124191461837	3115	0.503719275549806	Sig
KAISO_HUMAN.H11MO.1.A	1.840030567469271e-86	-85.73517	2888	0.4670116429495472	1826	0.29527813712807244	Sig
MLX_HUMAN.H11MO.0.D	8.11901614846211e-82	-81.0905	4744	0.7671410090556274	5535	0.8950517464424321	Sig
ALX3_HUMAN.H11MO.0.D	3.2897559701277345e-16	-15.48284	6100	0.9864165588615783	5960	0.963777490297542	Sig
GATA4_MOUSE.H11MO.0.A	0.0009722257333249527	-3.0122

In [17]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

# Due to the ISL1 dataset's large size, the following command may take a while since it runs 5 loops of the command 
# with a best of 5 to calculate average runtime. Feel free to revisit this notebook in a few minutes.
printf 'Runtime for pyxis with ISL1 peaks dataset:\n'
python -m timeit -s 'import os' 'os.system("pyxis analysis/GSM5838045_S1_ISL1_WT_d6CP_Rep1.bed benchmark/hg19.fa benchmark/isl1.pwms")'

Runtime for pyxis with ISL1 peaks dataset:
1 loop, best of 5: 341 sec per loop


### Comparing Runtime with HOMER's findMotifsGenome.pl

In [22]:
%%bash
# HOMER output for the test files in pyxis
python -m timeit -s 'import os' 'os.system("findMotifsGenome.pl example-files/peaks.bed example-files/ref.fa homer-results/example-data -size given -len 11 > /dev/null")'

1 loop, best of 5: 186 sec per loop


In [33]:
%%bash
# For the ISL1 TF dataset. Note: This command will around an hour to run.
python -m timeit -r 1 -s 'import os' 'os.system("findMotifsGenome.pl analysis/GSM5838045_S1_ISL1_WT_d6CP_Rep1.bed benchmark/hg19.fa homer-results/isl1 > /dev/null")'

1 loop, best of 1: 3.12e+03 sec per loop


#### Summary

Runtime per loop of basic usage commands for `pyxis` vs. `HOMER` is as follows:

| Tool | Dataset | Runtime Per Loop (1 loop, best of 5) |
| :--- | :------ | :----------------------------------: |
| Pyxis | Example Data | 1.81 sec |
| Pyxis | ISL1 | 341 sec |
| HOMER | Example Data | 186 sec |
| HOMER | ISL1 | 3.12e+03 sec |

In [31]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

printf 'Basic Usage Command:\n'
python -m timeit -s 'import os' 'os.system("pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms > /dev/null")'

Basic Usage Command:
1 loop, best of 5: 1.81 sec per loop


## II. Memory Consumption

In [32]:
!git clone https://github.com/jhclark/memusg.git

fatal: destination path 'memusg' already exists and is not an empty directory.


In [33]:
pwd # check that you are still in the pyxis directory

'/home/esquan/final_project/pyxis'

IMPORTANT: Only run the below cell __once__ or you will encounter an error in the following cells.

In [34]:
%%bash
# Moving the memusg script to the pyxis directory for execution
mv memusg/ memusage/
cd memusage
mv memusg ..
cd ..
rm -rf memusage
export PATH=.

mv: cannot stat 'memusg/': Not a directory
mv: cannot stat 'memusg': No such file or directory


### Testing with Example Files Data

In [27]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

# Testing pyxis memory usage
# 1. Basic usage command
./memusg pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms > /dev/null

# 2. Including a background peaks file
./memusg pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms -b example-files/background.bed > /dev/null

# 3. Including background + generation of sequence logos
./memusg pyxis example-files/peaks.bed example-files/ref.fa example-files/test.pwms -b example-files/background.bed -s > /dev/null

Background not specified. Generating background sequences from reference genome...
Calculating thresholds for each PWM...
[1/5] .... 
[2/5] .... 
[3/5] .... 
[4/5] .... 
[5/5] .... Done.

Performing motif enrichment...
[1/5] .... 
[2/5] .... 
[3/5] .... 
[4/5] .... 
[5/5] .... Done.

Creating output file 'enrichment_results.tsv'...
....
....
....
....
.... Done.


Pyxis has run successfully, please check 'pyxis_enrichments.tsv' for motif enrichment info!
If '-s' or '--seqlogo' was specified, the motif sequence logos should be in your directory.



memusg: vmpeak: 17959672 kb
memusg: vmpeak: 17959676 kb
memusg: vmpeak: 17959680 kb


In [35]:
%%bash
# Testing findMotifsGenome.pl memory usage
./memusg findMotifsGenome.pl example-files/peaks.bed example-files/ref.fa homer-results/example-data > /dev/null

memusg: vmpeak: 167048 kb


### Testing Memory Usage with ISL1 Dataset

In [39]:
%%bash
DIR=`cat scriptpath.txt`
export PATH=$PATH:$DIR

# pyxis
./memusg pyxis analysis/GSM5838045_S1_ISL1_WT_d6CP_Rep1.bed benchmark/hg19.fa benchmark/isl1.pwms > /dev/null
# HOMER
currdir=$(pwd)
./memusg findMotifsGenome.pl $currdir'/analysis/GSM5838045_S1_ISL1_WT_d6CP_Rep1.bed' $currdir'/benchmark/hg19.fa' homer-results/isl1 > /dev/null

memusg: vmpeak: 17966576 kb
memusg: vmpeak: 1146152 kb


#### Summary

Memory Usage between `pyxis` vs. `HOMER` is as follows:

| Tool | Dataset | Command | Vmpeak |
| :--- | :------ | :------ | :----: |
| Pyxis | Test Data | Basic Usage | 17959672 kb |
| Pyxis | Test Data | Basic Usage + Background | 17959676 kb |
| Pyxis | Test Data | Basic Usage + Background + Seqlogo | 17959680 kb |
| Pyxis | ISL1 | Basic Usage | 17966576 kb |
| HOMER | Test Data | Basic Usage | 167048 kb |
| HOMER | ISL1 | Basic Usage | 1146152 kb |